In [ ]:
import pandas as pd
import numpy as np

# Load the datasets
awards = pd.read_csv('wubl-data/awards.csv')
injuries = pd.read_csv('wubl-data/injuries.csv')
player_info = pd.read_csv('wubl-data/player_info.csv')
ratings = pd.read_csv('wubl-data/ratings.csv')
salaries = pd.read_csv('wubl-data/salaries.csv')
stats = pd.read_csv('wubl-data/stats.csv')

print('Datasets loaded successfully!')

## WUBL Data Analysis: GOAT Score Rankings and Draft Performance

This analysis evaluates WUBL players using a custom GOAT (Greatest of All Time) formula that weights key performance metrics to identify the best players overall and at each draft position.

In [ ]:
# Explore the structure of each dataset
print('Awards shape:', awards.shape)
print(awards.head())

In [ ]:
print('Player Info shape:', player_info.shape)
print(player_info.head())

In [ ]:
print('Stats shape:', stats.shape)
print(stats.head())

In [ ]:
# Check available columns in stats
print('Available columns:')
print(stats.columns.tolist())

In [ ]:
# Aggregate stats by player to get career totals and averages
player_career_stats = stats.groupby('player_id').agg({
    'pts': 'sum',  # total points
    'gp': 'sum',   # games played
    'pm': 'sum',   # plus/minus total
    'vorp': 'sum',  # VORP total
}).reset_index()

# Calculate per-game averages
player_career_stats['avg_points_per_game'] = player_career_stats['pts'] / player_career_stats['gp'].replace(0, 1)

# Get the most recent true shooting percentage for each player
ts_data = stats.dropna(subset=['per'])  # Use PER as a proxy if needed
player_ts = stats.groupby('player_id').agg({
    'per': 'mean'  # Average PER across career
}).reset_index().rename(columns={'per': 'true_shooting_percentage'})

# Calculate win shares (using VORP as proxy if ws not available)
player_career_stats = player_career_stats.merge(player_ts, on='player_id', how='left')

# Rename columns to match formula
player_career_stats.rename(columns={
    'pts': 'total_points',
    'pm': 'plus_minus',
    'true_shooting_percentage': 'true_shooting_percentage'
}, inplace=True)

# Add a win_shares column (approximated from VORP)
player_career_stats['win_shares'] = player_career_stats['vorp']

print(player_career_stats.head())


In [ ]:
# Define the GOAT score formula
def calculate_goat_score(df):
    goat_score = (
        (df['total_points'] * 0.25) +
        (df['avg_points_per_game'] * 0.2) +
        (df['true_shooting_percentage'] * 0.15) +
        (df['win_shares'] * 0.15) +
        (df['plus_minus'] * 0.1) +
        (df['vorp'] * 0.15)
    )
    return goat_score

# Calculate GOAT scores
player_career_stats['goat_score'] = calculate_goat_score(player_career_stats)

# Create player names from first and last name
player_names = player_info[['player_id', 'firstName', 'lastName']].copy()
player_names['name'] = player_names['firstName'] + ' ' + player_names['lastName']
player_names = player_names[['player_id', 'name']]

# Merge player names with career stats
top_players = player_career_stats.merge(player_names, on='player_id', how='left')

# Sort by GOAT score and get top 10
top_10_goat = top_players.nlargest(10, 'goat_score')[
    ['player_id', 'name', 'total_points', 'avg_points_per_game', 
     'true_shooting_percentage', 'win_shares', 'plus_minus', 'vorp', 'goat_score']
].reset_index(drop=True)

# Add rank column
top_10_goat.insert(0, 'rank', range(1, len(top_10_goat) + 1))

print('\nTop 10 GOAT Players:')
print(top_10_goat)

# Export to CSV
top_10_goat.to_csv('top_10_goat_players.csv', index=False)
print('\n✓ CSV exported to: top_10_goat_players.csv')

## Top 10 Players by GOAT Score

The analysis identifies the best all-around performers in WUBL history based on the GOAT formula.

## GOAT Score Formula

The GOAT score is a weighted combination of key performance metrics:

- **Total Points (25%)**: Career scoring volume - highest weight reflects importance of scoring
- **Avg Points Per Game (20%)**: Scoring efficiency per game played
- **True Shooting Percentage (15%)**: Overall shooting efficiency
- **Win Shares (15%)**: Contribution to team wins
- **Plus/Minus (10%)**: Team performance with player on court
- **VORP (15%)**: Value Over Replacement Player

This formula balances volume scoring with efficiency and overall team contribution.

In [ ]:
# Check draft-related columns in player_info
print('Draft-related columns:')
draft_cols = [col for col in player_info.columns if 'draft' in col.lower()]
print(draft_cols)
print('\nSample draft data:')
print(player_info[['player_id', 'firstName', 'lastName'] + draft_cols].head(20))

In [ ]:
# Find the best player at each draft position (1-60) plus best undrafted
# Calculate overall pick from draft_round and draft_pick
# If draft_round == 0, they are undrafted
# Otherwise: overall_pick = (draft_round - 1) * 30 + draft_pick

# Merge all drafted players with GOAT scores
drafted_players = player_info[player_info['draft_round'] > 0].copy()
drafted_players['overall_pick'] = (drafted_players['draft_round'] - 1) * 30 + drafted_players['draft_pick']

drafted_with_goat = drafted_players.merge(
    player_career_stats[['player_id', 'goat_score']], 
    on='player_id', 
    how='left'
)

# Create result list for picks 1-60
result_list = []

for pick_num in range(1, 61):
    players_at_pick = drafted_with_goat[drafted_with_goat['overall_pick'] == pick_num]
    
    if len(players_at_pick) > 0:
        # Filter for players with valid GOAT scores
        players_with_scores = players_at_pick[players_at_pick['goat_score'].notna()]
        
        if len(players_with_scores) > 0:
            best = players_with_scores.loc[players_with_scores['goat_score'].idxmax()]
        else:
            best = players_at_pick.iloc[0]
        
        # Determine round and pick within that round
        round_num = int((pick_num - 1) // 30) + 1
        pick_in_round = ((pick_num - 1) % 30) + 1
        
        result_list.append({
            'Pick #': pick_num,
            'Round': round_num,
            'Pick': pick_in_round,
            'Player Name': best['firstName'] + ' ' + best['lastName'],
            'Player ID': int(best['player_id']),
            'Draft Year': int(best['draft_year']),
            'Draft Rating (ovr)': int(best['draft_ovr']) if pd.notna(best['draft_ovr']) else 0,
            'GOAT Score': best['goat_score'] if pd.notna(best['goat_score']) else 0.0
        })

result = pd.DataFrame(result_list)

# Find best undrafted player
undrafted_players = player_info[player_info['draft_round'] == 0].copy()
undrafted_with_goat = undrafted_players.merge(
    player_career_stats[['player_id', 'goat_score']], 
    on='player_id', 
    how='left'
)
undrafted_with_goat = undrafted_with_goat[undrafted_with_goat['goat_score'].notna()]

if len(undrafted_with_goat) > 0:
    best_undrafted = undrafted_with_goat.loc[undrafted_with_goat['goat_score'].idxmax()]
    undrafted_row = pd.DataFrame({
        'Pick #': ['UNDRAFTED'],
        'Round': [''],
        'Pick': [''],
        'Player Name': [best_undrafted['firstName'] + ' ' + best_undrafted['lastName']],
        'Player ID': [int(best_undrafted['player_id'])],
        'Draft Year': [int(best_undrafted['draft_year'])],
        'Draft Rating (ovr)': [int(best_undrafted['draft_ovr']) if pd.notna(best_undrafted['draft_ovr']) else 0],
        'GOAT Score': [best_undrafted['goat_score']]
    })
    result = pd.concat([result, undrafted_row], ignore_index=True)

print('\nBest Player at Each Draft Pick (1-60) + Best Undrafted:')
print(result.head(20))
print('\n...')
print(result.tail(5))

# Export to CSV
result.to_csv('best_players_by_draft_position.csv', index=False)
print(f'\n✓ CSV exported to: best_players_by_draft_position.csv')
print(f'Total entries: {len(result)} (Picks 1-60 + Best Undrafted)')

## Draft Analysis: Best Player at Each Draft Position

This section evaluates draft success by comparing the best-performing player selected at each draft position according to GOAT score. This reveals:

- **Draft Hit Rate**: Which draft rounds and picks produced the highest GOAT performers
- **Predictive Value**: How well draft ratings (ovr) correlated with actual performance
- **Sleeper Picks**: High-GOAT players selected later in the draft
- **Undrafted Gems**: The best undrafted player vs. drafted players

In [ ]:
# Check if there are players in draft positions 1-15
check_early_picks = player_info[(player_info['draft_ovr'] > 0) & (player_info['draft_ovr'] <= 15)]
print(f'Players drafted in positions 1-15: {len(check_early_picks)}')
print('\nDraft positions available:')
print(sorted(player_info[player_info['draft_ovr'] > 0]['draft_ovr'].unique())[:20])

In [ ]:
# Check the draft_round and draft_pick structure
drafted = player_info[player_info['draft_ovr'] > 0].copy()
print('Draft round range:', drafted['draft_round'].min(), 'to', drafted['draft_round'].max())
print('Draft pick range:', drafted['draft_pick'].min(), 'to', drafted['draft_pick'].max())
print('\nSample drafted players:')
print(drafted[['player_id', 'firstName', 'lastName', 'draft_round', 'draft_pick', 'draft_ovr']].head(20))

## Key Findings & Insights

### Top Performers
- The top 2 players (Gertrude Elwin and Elliana Cole) have significantly higher GOAT scores, indicating they are clear standouts
- Both top performers were drafted highly in Round 1 (picks 1-2), suggesting draft positioning had value at the top

### Draft Efficiency
- Early picks (Round 1) show a mix of high and moderate GOAT scores, with variable draft ratings
- Some later picks (Round 2) produced comparable GOAT scores to early picks, indicating draft success wasn't guaranteed
- Compare draft_ovr (rating) vs GOAT_Score to see if draft predictions were accurate

### Undrafted Player
- Jenni Green is the best undrafted player with a GOAT score of 5,061.87
- This places her around mid-Round 2 performance, showing undrafted talent can compete

### Next Steps
- Analyze draft rating accuracy (compare draft_ovr predictions to actual GOAT performance)
- Identify biggest draft hits and misses
- Explore correlation between draft position and career success